# 8章　Tree-Based Methods

R 3.6.0を使用しています。

In [1]:
# Data
rm( list = ls( envir = globalenv() ), envir = globalenv() )


## 1. The Basics of Decision Trees

### 1. Regression Trees

regression treesの構築方法は，大まかに次の2ステップである。

1. 説明変数$X_1, X_2, \dots, X_p$を，重複がないように$J$個の領域$R_1, R_2, \dots, R_J$に切り分ける。
2. すべての$R_j$について，観測が$R_j$に属する場合は，$R_j$に属する訓練データの平均値を予測値として返す。

$R_j$は，RSS（残差二乗和）が最小になるようにする。
$$
\sum_{j=1}^J\sum_{i\in R_j}(y_i - \hat{y}_{R_j})^2, \tag{8.1}
$$
ここで，$\hat{y}_{R_j}$は，$R_j$に属する訓練データの平均値（すなわち，決定木による予測値）である。

具体的には，
$$
R_1(j, s) = \{ X|X_j < s \} \ and \ R_2(j, s) = \{ X|X_j \geq s \}, \tag{8.2}
$$
となるsを見つけるということ，つまり，
$$
\sum_{i: x_i \in R_1(j, s)}(y_i - \hat{y}_{R_1})^2 + \sum_{i: x_i \in R_2(j, s)}(y_i - \hat{y}_{R_2})^2, \tag{8.3}
$$
を最小化させるjとsの値を見つければよい。

### Tree Pruning（決定木の剪定）

上記の方法では結果が複雑になりすぎるので，バイアスを少し大きくする代わりに，分散が小さく，解釈しやすい小さい木にしたい。  
そこで，Cost complexity pruningまたはweakest link pruningと呼ばれる，木の剪定アルゴリズムがある。
$$
\sum_{m=1}^{|T|} \sum_{x_i \in R_m} (y_i - \hat{y}_{R_m})2 + \alpha |T| \ tag{8.4}
$$
ここで，$T \subset T_0$で，$\alpha \geq 0$である。また，$|T|$は$T$の終端ノードの個数である。この（8.4）式をなるべく小さくすればよい。

さらに，木の剪定アルゴリズムの手順は次の通りである。

1. 各終端ノードの中の観測データの個数が一定数以下になるまで領域に分割する。
2. $\alpha$の関数として，cost complexity pruningを適用する。
3. $\alpha$を選ぶために，K-fold validationを使う。$1 \sim K$の$K$それぞれに対し，  
    (a)ステップ1と2を，k番目のトレーニングデータ以外に適用する。  
    (b)$\alpha$の関数として，k番目のデータの二乗平均予測誤差を求める。  
    $\alpha$の平均をとり，平均誤差を最小化させる$\alpha$を選択する。
4. 選んだ$\alpha$をステップ2の関数に代入し，部分木を返す。

### 2. Classification Trees



**参考文献**
kfumix, [ ISLR: Chapter 8 Tree-Based Methods ](http://kfumix.hatenablog.com/entry/2017/06/25/212754) (2019/6/4)